# jEETENGE TO HUM HI by gaurav

### Load libraries and data

In [1]:
import pandas as pd
import numpy as np
# import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
print("5train shape", train.shape)
print("test shape", test.shape)

5train shape (12137810, 10)
test shape (3706907, 9)


In [3]:
# check missing values per column
train.isnull().sum(axis=0)/train.shape[0]

ID             0.000000
datetime       0.000000
siteid         0.099896
offerid        0.000000
category       0.000000
merchant       0.000000
countrycode    0.000000
browserid      0.050118
devid          0.149969
click          0.000000
dtype: float64

In [4]:
train.click.value_counts()/train.shape[0]

0    0.963979
1    0.036021
Name: click, dtype: float64

In [5]:
train[train.click==1].shape

(437214, 10)

### Clean Data and Create Features

In [6]:
train['siteid'].fillna(-999, inplace=True)
test['siteid'].fillna(-999, inplace=True)

train['browserid'].fillna("None", inplace=True)
test['browserid'].fillna("None", inplace=True)

train['devid'].fillna("None", inplace=True)
test['devid'].fillna("None", inplace=True)

In [7]:
# set datatime
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

In [8]:
# create datetime variable
train['tweekday'] = train['datetime'].dt.weekday
train['thour'] = train['datetime'].dt.hour
train['tminute'] = train['datetime'].dt.minute

test['tweekday'] = test['datetime'].dt.weekday
test['thour'] = test['datetime'].dt.hour
test['tminute'] = test['datetime'].dt.minute

In [9]:
cols = ['siteid','offerid','category','merchant']

for x in cols:
    train[x] = train[x].astype('object')
    test[x] = test[x].astype('object')

In [10]:
cat_cols = cols + ['countrycode','browserid','devid']

In [11]:
for col in cat_cols:
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values) + list(test[col].values))
    train[col] = lbl.transform(list(train[col].values))
    test[col] = lbl.transform(list(test[col].values))

In [12]:
train.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click,tweekday,thour,tminute
0,IDsrk7SoW,2017-01-14 09:42:09,128865,784773,48,127,4,2,2,0,5,9,42
1,IDmMSxHur,2017-01-18 17:50:53,142053,157563,59,65,1,8,0,0,2,17,50
2,IDVLNN0Ut,2017-01-11 12:46:49,2618,458279,69,15,0,1,2,0,2,12,46
3,ID32T6wwQ,2017-01-17 10:18:43,243406,345067,117,507,2,2,1,0,1,10,18
4,IDqUShzMg,2017-01-14 16:02:33,154278,417948,36,276,3,8,0,0,5,16,2


### Model Training

In [13]:
cols_to_use = list(set(train.columns) - set(['ID','datetime','click']))

In [14]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=20)
a = 0
for i, j in skf.split(train[cols_to_use], train['click']):
    a = j
len(a)

606889

In [15]:
X_train, X_test, y_train, y_test = train_test_split(train.loc[a][cols_to_use], train.loc[a]['click'], test_size = 0.5)

In [ ]:
y_test.value_counts()

0    292444
1     11001
Name: click, dtype: int64

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import time
t1 = time.time()
dict1 = {}
dict2 = {}
for i in range(300, 600, 10):
    
    print("Model with ", i, " n estimators")
    model_mod = RandomForestClassifier(n_estimators=i, verbose=1, oob_score=True, n_jobs=-1, random_state=42)#lets use the fucing cores
    start = time.time()
    model_mod.fit(X_train, y_train)
    print("time to fit: ", time.time()-start)
    start = time.time()
    pred = model_mod.predict(X_test)
    print("time to predict: ", time.time()-start ,"\n")
    print("training score: ", roc_auc_score(y_train, model_mod.predict(X_train)))
    print("crossval score: ", roc_auc_score(y_test, pred))
    print("oob score: ", model_mod.oob_score_)
    dict1[i]=roc_auc_score(y_test, pred)
    dict2[i]=model_mod.oob_score_
    print("Done :: ", i)
print("total time ", time.time()-t1)

Model with  300  n estimators


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   29.2s finished


time to fit:  39.16286563873291


[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    1.0s
[Parallel(n_jobs=12)]: Done 300 out of 300 | elapsed:    1.8s finished


time to predict:  2.3157787322998047 



...

[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    1.0s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    2.5s
[Parallel(n_jobs=12)]: Done 540 out of 540 | elapsed:    3.3s finished


time to predict:  4.062948942184448 



[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    1.0s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    2.6s
[Parallel(n_jobs=12)]: Done 540 out of 540 | elapsed:    3.3s finished


training score:  1.0
crossval score:  0.798908145928
oob score:  0.977844346898
Done ::  540
Model with  550  n estimators


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   41.4s
[Parallel(n_jobs=-1)]: Done 550 out of 550 | elapsed:   52.9s finished


400, njob=-1
training score:  1.0
crossval score:  0.799803514688
oob score:  0.978233216013

0    294387
1      9058
Name: 0, dtype: int64
0    292421
1     11024
Name: click, dtype: int64

600, njob=-1
worse performance

470, 510 looks good to me

In [ ]:
pred = pd.DataFrame(pred)
print(pred[0].value_counts())
y_test = pd.Series(y_test)
print(y_test.value_counts())

In [ ]:
model.random_state

In [ ]:
model.

In [ ]:
a = test.columns

In [ ]:
pred = model_mod.predict_proba(test[cols_to_use])

In [ ]:
pred = pd.DataFrame(pred)
pred.shape

In [ ]:
model.feature_importances_

In [ ]:
sub = pd.DataFrame({'ID':test['ID'], 'click':pred[1]})
# sub.to_csv('smallData_rf_1.csv', index=False)